In [1]:
import pandas as pd
import numpy as np

final_df = pd.DataFrame()
#joining the 7 datasets from web scraping together
for k in ['California', 'Washington', 'Oregon', 'Idaho', 'Nevada', 'Arizona', 'Utah']:
    df = pd.read_csv(f'Datasets_WebScraping/dataset_{k}.csv')
    #remvoing unnecessary columns, not important for analysis
    df_2 = df.drop(columns=['externalApplyLink','descriptionHTML', 'isExpired', 'postedAt', 'urlInput', 'postingDateParsed', 'scrapedAt','rating', 'reviewsCount', 'searchInput/country', 'url', 'urlInput', 'searchInput/position', 'jobType'])
    df_2 = df_2.dropna(subset=['salary'])
    final_df = pd.concat([final_df, df_2], ignore_index=True)
    print(len(df_2))
    print(f'With {k} the total is {len(final_df)}')

#removing duplicates and empty columns
final_df = final_df.drop_duplicates()
final_df = final_df.drop(columns=['jobType/3', "jobType/4"])
final_df = final_df.reset_index(drop=True)

769
With California the total is 769
545
With Washington the total is 1314
74
With Oregon the total is 1388
27
With Idaho the total is 1415
54
With Nevada the total is 1469
105
With Arizona the total is 1574
83
With Utah the total is 1657


In [2]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1656 entries, 0 to 1655
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   company               1656 non-null   object
 1   description           1656 non-null   object
 2   id                    1656 non-null   object
 3   jobType/0             1226 non-null   object
 4   jobType/1             118 non-null    object
 5   jobType/2             16 non-null     object
 6   location              1656 non-null   object
 7   positionName          1656 non-null   object
 8   salary                1656 non-null   object
 9   searchInput/location  1656 non-null   object
dtypes: object(10)
memory usage: 129.5+ KB


In [3]:
final_df.head(10)

,company,description,id,jobType/0,jobType/1,jobType/2,location,positionName,salary,searchInput/location
0,Providence,Description\n Providence is calling a Senior D...,e14d7653c80fe244,Full-time,NaN,NaN,California,Senior Data Scientist,$45.56 - $100.22 an hour,California
1,Slalom Consulting,Who You’ll Work With\n As a modern technology ...,80a2597defd8f97c,NaN,NaN,NaN,"Orange County, CA",Slalom Flex (Project Based) - Sr. Data Analyst,$65 - $95 an hour,California
2,Gradient,About the Company\n Gradient is on a mission t...,2d98b8b36444c36c,Full-time,NaN,NaN,"Burlingame, CA",Founding AI Strategist,"$120,000 - $200,000 a year",California
3,Edwards Lifesciences,Imagine how your ideas and expertise can chang...,e0f49e777f057ff8,Full-time,NaN,NaN,California,"Principal, Statistical Programming Remote","$126,000 - $178,000 a year",California
4,Kittelson & Associates,Location: Oakland Orange Sacramento San Diego ...,6cc4dd9be8bd8ffa,NaN,NaN,NaN,"Oakland, CA",Senior Engineer or Planner with Focus on Trave...,"$120,000 - $150,000 a year",California
5,Tesla,Job Category\n \n \n AI & R...,378be55af93df555,Full-time,NaN,NaN,"Palo Alto, CA","AI Research Engineer, Data Scaling, Self-Driving","$140,000 - $360,000 a year",California
6,Hewlett Packard Enterprise,Hewlett Packard Labs - Machine Learning Resear...,31e55d3f70f75ce0,Full-time,NaN,NaN,"Milpitas, CA 95035",Hewlett Packard Labs - Machine Learning Resear...,"$117,500 - $270,000 a year",California
7,"TrellisWare Technologies, Inc - Recent Grads",TrellisWare launched in 2000 with an innovativ...,8c464d9bd06cfc6f,Full-time,NaN,NaN,"San Diego, CA","Systems (Communications, Signal Processing, Ma...","$80,000 - $100,000 a year",California
8,Axos Bank,"Axos Bank\n \n \n Target Range:\n $80,000.00...",6a689ee569a99dcc,Full-time,NaN,NaN,"4350 La Jolla Village Drive, Suite 140, San Di...","Analytics Engineer, Data Analytics","$80,000 - $100,000 a year",California
9,Lawrence Livermore National Laboratory,Company Description Join us and make YOUR mar...,364ec17f286ea836,Full-time,NaN,NaN,"Livermore, CA",Machine Learning - Academic Graduate Appointee,"$6,692.90 - $7,650.50 a month",California


In [3]:
#converting types of objects to strings to avoid problems later on 
final_df['salary'] = [str(i) for i in final_df['salary']]
final_df['description'] = [str(i) for i in final_df['description']]
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1656 entries, 0 to 1655
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   company               1656 non-null   object
 1   description           1656 non-null   object
 2   id                    1656 non-null   object
 3   jobType/0             1226 non-null   object
 4   jobType/1             118 non-null    object
 5   jobType/2             16 non-null     object
 6   location              1656 non-null   object
 7   positionName          1656 non-null   object
 8   salary                1656 non-null   object
 9   searchInput/location  1656 non-null   object
dtypes: object(10)
memory usage: 129.5+ KB


### JOB POSITION NAMES AND SENIORITY LEVELS

In [4]:
#importing libraries for nlp
import spacy
import re

In [6]:
import gensim.downloader as api
word2vec_model = api.load('word2vec-google-news-300')

In [7]:
from difflib import get_close_matches
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
nlp = spacy.load("en_core_web_sm")

In [8]:
def preprocess_text(text):
    doc = nlp(text.lower())
    #removing stopwords and chracters that are not letters
    cleaned_tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    bigram_measures = BigramAssocMeasures()
    #group the tokens into bigrams
    finder = BigramCollocationFinder.from_words(cleaned_tokens)
    #searching for the most related 1000 bigrams
    top_bigrams = finder.nbest(bigram_measures.pmi, 1000)
    return top_bigrams

def preprocess_label(text):
    doc = nlp(text.lower())
    cleaned_tokens = [token.lemma_ for token in doc]
    return cleaned_tokens

def similarity_w2v(tokens1, tokens2):
    #checking if the tokens are contained in the world2vec model 
    for i in tokens1:
        if not word2vec_model.has_index_for(i):
            return -1
    for i in tokens2:
        if not word2vec_model.has_index_for(i):
            return -1
    #transforming the tokens into vectors
    vector1 = np.mean([word2vec_model[word] for word in tokens1], axis=0)
    vector2 = np.mean([word2vec_model[word] for word in tokens2], axis=0)
    # Computing the cosine similarity between the two vectors
    similarity = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    
    return similarity

def find_similar_job_pos(job_position, categories, seniority, threshold):
    # Preprocess job description
    tokens = preprocess_text(job_position)
    #find the words in job position that best match a given set of categories saving them to mathc list 
    matches = set()
    for key, vals in categories.items():
        for val in vals:
            pos_tokens = preprocess_label(val)
            for token in tokens:
                # ditinguish between 2 possibilities:
                # categories names or words with same root are present in text
                # categories are not present in position name so that category with cosine similarity higher than 0.4,
                # (to select this value we have tried many values and seen that this produces the best reults) is added to matches 
                #print(token, subject_tokens)
                if token in pos_tokens:
                    matches.add((key, 1))
                elif get_close_matches(token, [pos_tokens], n=1, cutoff=threshold):
                    matches.add((key, threshold))
                elif similarity_w2v(token, pos_tokens) > 0.4: 
                    #print(token, pos_tokens)
                    matches.add((key, similarity_w2v(token, pos_tokens)))
    #extrcating seniority levels by same procedure above
    matches2 = set()
    for key, words in seniority.items():
        for word in words:
            seniority_tokens = preprocess_label(word)
            for token in tokens:
                if token in seniority_tokens:
                    matches2.add(key)
                elif get_close_matches(token, [seniority_tokens], n=1, cutoff=threshold-0.3):
                    matches2.add(key)
    #select category with highest similarity in matches and return it if present
    if matches2:
        max_sen = max(matches2, key=lambda x: x[1])
    else:
        max_sen = "Not specified"
    #same as above
    if matches:
        max_pos = max(matches, key=lambda x: x[1])
        pos, sim = max_pos
        return [pos, sim, max_sen]
    else:
        max_pos = "Other"
        return ["Other", 0, max_sen]

In [9]:
#dictionaries with lists of words used for matching job position names and seniority to the corresponding categories
seniority_list = {
    "Senior": ["senior", "Sr", "Superior", "Experienced"],
    "Junior": ["Jr", "Entry-level","Associate","Novice", "Apprentice"]
}

job_pos_list = {
    "Data analyst": ["Data analyst", "Data Specialist",
        "Data Scientist",
        "Business Analyst",
        "Data Consultant",
        "Data Interpreter",
        "BI Analyst",
        "Reporting Analyst",
        "Quantitative Analyst",
        "Market Analyst",
        "Operations Analyst"],
    "Software Engineer": ["Software Engineer",
        "Software Developer",
        "Programmer",
        "Backend Developer",
        "Frontend Developer",
        "Full Stack Developer",
        "Application Developer",
        "System Architect",
        "Coder",
        "DevOps Engineer",
        "Software Architect"
    ],
    "Researcher": ["Researcher","Scientist",
        "Academic",
        "Research Analyst",
        "Scholar",
        "Postdoctoral Fellow"],
    "ML/AI Engineer": [
        "Machine Learning Engineer",
        "Artificial Intelligence Engineer",
        "Deep Learning Engineer",
        "AI Developer",
        "NLP Engineer",
        "Computer Vision Engineer",
        "AI Specialist",
        "AI Architect",
        "Predictive Modeler"
    ],
    "Manager":[
        "Team Leader",
        "Supervisor",
        "Coordinator",
        "Administrator",
        "Project Manager",
        "Operations Manager",
        "Department Head",
        "Director",
        "Executive",
        "Managerial Officer"
    ],
    "Statistician":["Statistician",
        "Biostatistician",
        "Econometrician",
        "Statistical Analyst",
        "Risk Analyst",
        "Actuary",
        "Mathematical Analyst"
    ],
    "Sales":["Salesperson",
        "Business Development Executive",
        "Sales Consultant",
        "Client Relationship",
        "Sales Representative",
        "Inside Sales Representative",
        ],
    "Cloud ops":["Cloud Engineer",
        "Cloud Administrator",
        "DevOps Engineer",
        "Site Reliability Engineer (SRE)",
        "Infrastructure Engineer",
        "Cloud Architect",
        "Platform Engineer",
        "Cloud Operations Specialist",
        "Systems Engineer",
        "IT Administrator"],
    }

In [14]:
find_similar_job_pos(final_df['positionName'][1655], job_pos_list, seniority_list, threshold=0.9)

['Software Engineer', 0.74800867, 'Senior']

In [15]:
#extract the position names and seniority levels for all rows of the dataset applying the function above
extracted_job_positions = []
for i in range(len(final_df['positionName'])):
    text = final_df['positionName'].iloc[i]
    job_inter = find_similar_job_pos(str(text), job_pos_list, seniority_list, threshold=0.9)
    extracted_job_positions.append(job_inter)

In [21]:
#analyse the extracted list of categories for job position names
count = [1 for i in range(len(extracted_job_positions)) if extracted_job_positions[i][1] < 0.6]
print(f'total count with low similarity: {sum(count)}')
null_indices = [index for index, sublist in enumerate(extracted_job_positions) if sublist[1] < 0.6]
print(f'indexes with low similarity: {null_indices}')

#analyse the extracted seniority levels 
count = [1 for i in range(len(extracted_job_positions)) if extracted_job_positions[i][2] == "Not specified"]
print(f'total count that are not specified: {sum(count)}')
null_indices = [index for index, sublist in enumerate(extracted_job_positions) if sublist[2] == "Not specified"]
print(f'indexes for not specified seniority: {null_indices}')

total count with low similarity: 146
indexes with low similarity: [3, 14, 16, 25, 48, 51, 89, 90, 112, 135, 140, 144, 161, 194, 197, 210, 225, 237, 266, 275, 303, 311, 330, 333, 348, 361, 370, 372, 388, 430, 459, 469, 484, 485, 587, 597, 610, 612, 621, 658, 666, 668, 684, 694, 699, 700, 706, 711, 724, 754, 761, 807, 822, 833, 836, 881, 889, 890, 891, 893, 898, 900, 904, 908, 909, 916, 930, 951, 965, 984, 991, 998, 1000, 1028, 1036, 1039, 1048, 1073, 1085, 1092, 1101, 1110, 1154, 1155, 1160, 1165, 1168, 1173, 1174, 1239, 1249, 1250, 1257, 1285, 1290, 1294, 1331, 1342, 1350, 1356, 1360, 1370, 1386, 1391, 1393, 1397, 1402, 1409, 1415, 1423, 1430, 1438, 1443, 1450, 1476, 1479, 1486, 1490, 1494, 1502, 1504, 1516, 1519, 1522, 1527, 1528, 1541, 1559, 1567, 1568, 1569, 1577, 1578, 1596, 1617, 1622, 1624, 1627, 1630, 1633, 1643, 1645, 1648, 1649, 1652, 1653]
total count that are not specified: 1131
indexes for not specified seniority: [2, 3, 5, 6, 7, 8, 9, 10, 12, 14, 15, 16, 18, 19, 20, 21, 22

In [22]:
sen_jtype_df = pd.DataFrame({"Jobtype": [i[0] for i in extracted_job_positions], "Seniority":[i[2] for i in extracted_job_positions]})
sen_jtype_df.describe()

,Jobtype,Seniority
count,1656,1656
unique,9,3
top,Data analyst,Not specified
freq,455,1131


In [23]:
final_df = pd.concat([final_df, sen_jtype_df], axis =1)
final_df.describe()

,company,description,id,jobType/0,jobType/1,jobType/2,location,positionName,salary,searchInput/location,Jobtype,Seniority
count,1656,1656,1656,1226,118,16,1656,1656,1656,1656,1656,1656
unique,590,1513,1655,8,7,5,445,1375,997,7,9,3
top,Amazon.com Services LLC,Outlier helps the world’s most innovative comp...,b2f361d276f3da9c,Full-time,Freelance,Freelance,"Seattle, WA",Data Scientist,"$136,000 - $222,200 a year",California,Data analyst,Not specified
freq,135,40,2,1094,64,10,251,31,44,768,455,1131


In [24]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1656 entries, 0 to 1655
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   company               1656 non-null   object
 1   description           1656 non-null   object
 2   id                    1656 non-null   object
 3   jobType/0             1226 non-null   object
 4   jobType/1             118 non-null    object
 5   jobType/2             16 non-null     object
 6   location              1656 non-null   object
 7   positionName          1656 non-null   object
 8   salary                1656 non-null   object
 9   searchInput/location  1656 non-null   object
 10  Jobtype               1656 non-null   object
 11  Seniority             1656 non-null   object
dtypes: object(12)
memory usage: 155.4+ KB


### Education extraction

In [25]:
#dictionary with categories of education and corresponding keywords 
education_dict = {
    "Associate's Degree": [
        "associate degree",
        "associate",
        "two-year degree",
        "AA degree",
        "AS degree"
    ],
    "Bachelor's Degree": [
        "bachelor's",
        "Bachelor's",
        "Bachelor's degree",
        "bachelor's degree",
        "Bachelor degree",
        "bachelor degree",
        "Bachelor",
        "bachelor",
        "undergraduate",
        "Undergraduate",
        "BSc", "B.Sc.",
        "BS", "bs", "B.S.",
        "baccalaureate",
    ],
    "Master's Degree": [
        "master's",
        "Master's",
        "Master's degree",
        "master's degree",
        "graduate degree",
        "postgraduate degree",
        "postgraduate",
        "Postgraduate",
        "Master",
        "master",
        "masters",
        "master degree",
        "Master degree",
        "MS",  "ms",
        "MA", "ma",
        "MSc", "M.Sc.",
        "ma degree",
    ],
    "Degree":["degree", "Degree", "degrees", "Degrees"],
    "Doctorate": [
        "Doctorate",
        "doctorate",
        "PhD",
        "Ph.D",
        "Ph.D.",
        "DPhil", "D.Phil.",
        "doctorate degree",
        "doctoral degree",
        "doctor of philosophy"
    ]
}

In [26]:
from spacy.pipeline import EntityRuler

# Load an empty English model
nlp = spacy.blank('en')

# Create an EntityRuler
ruler = nlp.add_pipe("entity_ruler")
#add patterns to the enttity ruler made of a list of keywords and the corresponding entity, education level using the above dictionary
patterns = []
for key in education_dict:
    for val in education_dict[key]:
        patterns.append({'label': key, 'pattern': val})
ruler.add_patterns(patterns)

#check that the patterns have been correctly added
print("Patterns in the EntityRuler:")
for pattern in patterns:
    print(pattern)

Patterns in the EntityRuler:
{'label': "Associate's Degree", 'pattern': 'associate degree'}
{'label': "Associate's Degree", 'pattern': 'associate'}
{'label': "Associate's Degree", 'pattern': 'two-year degree'}
{'label': "Associate's Degree", 'pattern': 'AA degree'}
{'label': "Associate's Degree", 'pattern': 'AS degree'}
{'label': "Bachelor's Degree", 'pattern': "bachelor's"}
{'label': "Bachelor's Degree", 'pattern': "Bachelor's"}
{'label': "Bachelor's Degree", 'pattern': "Bachelor's degree"}
{'label': "Bachelor's Degree", 'pattern': "bachelor's degree"}
{'label': "Bachelor's Degree", 'pattern': 'Bachelor degree'}
{'label': "Bachelor's Degree", 'pattern': 'bachelor degree'}
{'label': "Bachelor's Degree", 'pattern': 'Bachelor'}
{'label': "Bachelor's Degree", 'pattern': 'bachelor'}
{'label': "Bachelor's Degree", 'pattern': 'undergraduate'}
{'label': "Bachelor's Degree", 'pattern': 'Undergraduate'}
{'label': "Bachelor's Degree", 'pattern': 'BSc'}
{'label': "Bachelor's Degree", 'pattern': '

In [27]:
#function to extract education by looking for keywords
def extract_actual_edu(text):
    doc = nlp(text)
    ruled_ents = [str(ent.label_) for ent in doc.ents if ent.label_ in ["Associate's Degree", "Bachelor's Degree", "Master's Degree", "Degree", 'Doctorate']]
    ruled_ents = list(set(ruled_ents))
    return ruled_ents

extract_actual_edu(final_df["description"][0])

['Degree', "Master's Degree"]

In [28]:
#extract education for all job descriptions using function above and save it to a list 
extracted_education = []
for i in range(len(final_df['description'])):
    text = final_df['description'].iloc[i]
    edu_inter = extract_actual_edu(str(text))
    extracted_education.append(edu_inter)

In [32]:
count = [1 for i in extracted_education if i == []]
print(f'total null count: {sum(count)}')
null_indices = [index for index, sublist in enumerate(extracted_education) if sublist == []]
print(f'{null_indices}')

#check if the classes are balanced
masters = [1 for i in extracted_education if "Associate's Degree" in i ]
print(f'associate degrees: {sum(masters)}')

total null count: 217
[5, 12, 19, 25, 31, 36, 57, 62, 63, 74, 90, 96, 99, 108, 110, 122, 130, 132, 136, 145, 149, 157, 158, 171, 183, 184, 191, 196, 207, 218, 229, 230, 236, 242, 246, 253, 254, 266, 270, 277, 278, 291, 294, 296, 299, 302, 304, 334, 338, 351, 356, 357, 361, 362, 364, 369, 371, 374, 381, 382, 401, 404, 410, 425, 428, 439, 449, 453, 455, 482, 488, 490, 502, 505, 520, 530, 537, 541, 568, 573, 581, 587, 603, 607, 621, 623, 625, 626, 629, 633, 636, 640, 643, 648, 655, 659, 662, 676, 677, 689, 699, 700, 711, 727, 732, 739, 743, 763, 766, 771, 776, 785, 790, 796, 799, 803, 808, 824, 825, 826, 827, 828, 834, 840, 850, 851, 857, 865, 868, 870, 879, 882, 885, 890, 897, 906, 918, 926, 940, 945, 946, 949, 951, 961, 968, 994, 1001, 1002, 1014, 1033, 1038, 1050, 1054, 1065, 1069, 1083, 1090, 1098, 1107, 1110, 1113, 1122, 1125, 1146, 1172, 1179, 1186, 1197, 1201, 1204, 1208, 1215, 1229, 1264, 1267, 1271, 1273, 1274, 1277, 1284, 1288, 1292, 1298, 1299, 1303, 1308, 1319, 1326, 1351, 136

In [33]:
from spacy import displacy  
#check some cases to see whether the function works correctly
visual = final_df['description'][0]
doc = nlp(visual)
print(extracted_education[0])
print(extract_actual_edu(visual))
displacy.render(doc, style = 'ent')

['Degree', "Master's Degree"]
['Degree', "Master's Degree"]


In [34]:
# for each row of the dataset, the education levels are saved to different columns of a new dataframe,
# this involves creating k dummy variables for the k classes, which is usually incorrect, but is done only for
# type conversion issues of strings/lists faced when passing to the R code. Later the dummy variables are recombined into a single categorical variable
# later in the R analysis, the dummy variables will be combined back into a single categorical variable, preventing any mistakes
education_list = [i for i in education_dict]
education_final = np.zeros((len(final_df['id']), len(education_list)))
for i in range(education_final.shape[0]):
    for j in range(education_final.shape[1]):
        #print(i,j)
        if education_list[j] in extracted_education[i]:
            education_final[i][j] = 1

education_df = pd.DataFrame(education_final, columns=education_list)
education_df.head()

,Associate's Degree,Bachelor's Degree,Master's Degree,Degree,Doctorate
0,0.0,0.0,1.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,1.0,1.0,1.0
4,0.0,1.0,1.0,1.0,0.0


In [42]:
final_df = pd.concat([final_df, education_df], axis=1)
final_df.head()

,company,description,id,jobType/0,jobType/1,jobType/2,location,positionName,salary,searchInput/location,Jobtype,Seniority,Associate's Degree,Bachelor's Degree,Master's Degree,Degree,Doctorate
0,Providence,Description\n Providence is calling a Senior D...,e14d7653c80fe244,Full-time,NaN,NaN,California,Senior Data Scientist,$45.56 - $100.22 an hour,California,Researcher,Senior,0.0,0.0,1.0,1.0,0.0
1,Slalom Consulting,Who You’ll Work With\n As a modern technology ...,80a2597defd8f97c,NaN,NaN,NaN,"Orange County, CA",Slalom Flex (Project Based) - Sr. Data Analyst,$65 - $95 an hour,California,Data analyst,Senior,0.0,1.0,0.0,0.0,0.0
2,Gradient,About the Company\n Gradient is on a mission t...,2d98b8b36444c36c,Full-time,NaN,NaN,"Burlingame, CA",Founding AI Strategist,"$120,000 - $200,000 a year",California,ML/AI Engineer,Not specified,0.0,0.0,0.0,1.0,0.0
3,Edwards Lifesciences,Imagine how your ideas and expertise can chang...,e0f49e777f057ff8,Full-time,NaN,NaN,California,"Principal, Statistical Programming Remote","$126,000 - $178,000 a year",California,Statistician,Not specified,0.0,1.0,1.0,1.0,1.0
4,Kittelson & Associates,Location: Oakland Orange Sacramento San Diego ...,6cc4dd9be8bd8ffa,NaN,NaN,NaN,"Oakland, CA",Senior Engineer or Planner with Focus on Trave...,"$120,000 - $150,000 a year",California,Cloud ops,Senior,0.0,1.0,1.0,1.0,0.0


In [44]:
#final_df.drop(columns=["Associate's Degree", "Bachelor's Degree", "Master's Degree", "Degree", "Doctorate"], inplace = True)
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1656 entries, 0 to 1655
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   company               1656 non-null   object 
 1   description           1656 non-null   object 
 2   id                    1656 non-null   object 
 3   jobType/0             1226 non-null   object 
 4   jobType/1             118 non-null    object 
 5   jobType/2             16 non-null     object 
 6   location              1656 non-null   object 
 7   positionName          1656 non-null   object 
 8   salary                1656 non-null   object 
 9   searchInput/location  1656 non-null   object 
 10  Jobtype               1656 non-null   object 
 11  Seniority             1656 non-null   object 
 12  Associate's Degree    1656 non-null   float64
 13  Bachelor's Degree     1656 non-null   float64
 14  Master's Degree       1656 non-null   float64
 15  Degree               

### Experience extraction

In [45]:
#dicitonary for converting numbers from chracters to literal strings
word_to_num = {
    "zero": 0, "one": 1, "two": 2, "three": 3, "four": 4,
    "five": 5, "six": 6, "seven": 7, "eight": 8, "nine": 9,
    "ten": 10, "eleven": 11, "twelve": 12, "thirteen": 13,
    "fourteen": 14, "fifteen": 15
}

def extract_numbers(text):
    # search for all the numbers and digits in the text, job description
    digit_pattern = r"\b\d{1,2}\+?\b"
    digit_numbers = re.search(digit_pattern, text)

    if digit_numbers != None:
        return digit_numbers.group()
    else: 
        for word in text.lower().split():
            if word in word_to_num:
                return word_to_num[word]
#return the word representation of the number

In [46]:
#import the spacy model that was previsouly trained by us using 500/600 different labelled job descriptions with custom entity labels and trained only for NER
trained_nlp_2 = spacy.load('skills-detection-main\\trained_ner_model_2')
trained_nlp_2.pipe_names

c:\Users\flamo\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [47]:
def extract_exp(text):
    text = text.replace('\n', '')
    text = text.replace('\x0c', '')
    doc = trained_nlp_2(text) #tokenise the text and extract relevant entities
    exp_years = [ent.text for ent in doc.ents if ent.label_ in ['EXPERIENCE']]
    print(exp_years)
    exp_years2 = list(set([int(extract_numbers(k)) for k in exp_years]))
    if exp_years2 != []: #return the requried years of experience
        return max(exp_years2)
    if exp_years2 == []: #return 0 if no experience is found
        return 0

print(extract_exp(final_df['description'][8]))

['1 + years']
1


In [48]:
extracted_experience = [] #apply the function to the whole dataset and save the extracted years of experience to a list
for i in range(len(final_df['description'])):
    text = final_df['description'].iloc[i]
    exp_inter = extract_exp(str(text))
    #print(exp_inter)
    extracted_experience.append(exp_inter)
print(extracted_experience)

['6 years', '6 years', '6 years', '6 years']
['5+ years']
['4+ years']
['6 years', '5 years', '2 years']
[]
[]
[]
[]
['1 + years']
[]
['3+ years', '3+ years']
['3+ years']
[]
[]
['4+ years']
['5+ years']
[]
['6+ years', '3+ years']
[]
['10+ years']
['8 years', '6 years', '10 years', '8 years']
['3+ years', '4+ years']
[]
['five years', 'five years']
['5+ years']
['7 years']
['10 years', '8 years', '6 years', '8 years']
['1 year']
['3+ years', '6+ years']
['8 years', '6 years', '5+ years', '3+ years', '3+ years']
['4+ years', '2 years']
['8 years']
['4+ years', '2+ years']
['12+ months']
['10+ years', '5+ years', '5+ years']
['8 years', '5 years', '3 years', '5 years', '5 years', '3 years', '3 years', '8 years', '5 years', '3 years', '5 years', '5 years', '3 years', '3 years']
['10+ years', '10 years']
[]
['2+ years', '2+ years']
['12+ years']
['1+ years', '1+ years']
['5+ years', '2+ years']
['7+ years', '10+ years']
['4+ years', '8+ years', '6+ years']
['4+ years', '8+ years']
['12+ y

In [49]:
count = 0
for i in extracted_experience:
    if i == 0:
        count += 1
print(count) #almost all experiences are extracted
print(f'the number of rows for extracted experience is: {1657-count}')

#compare how many entries don't have either education or experience
null_count3 = 0
for i in range(len(extracted_experience)):
    if extracted_experience[i] == 0 and extracted_education[i] == 0:
        null_count3 += 1
print(f'the number of rows with both missing education and missing experience is: {null_count3}') 

431
the number of rows for extracted experience is: 1226
the number of rows with both missing education and missing experience is: 0


In [50]:
final_df["EXPERIENCE"] = extracted_experience
final_df.head()

,company,description,id,jobType/0,jobType/1,jobType/2,location,positionName,salary,searchInput/location,Jobtype,Seniority,Associate's Degree,Bachelor's Degree,Master's Degree,Degree,Doctorate,EXPERIENCE
0,Providence,Description\n Providence is calling a Senior D...,e14d7653c80fe244,Full-time,NaN,NaN,California,Senior Data Scientist,$45.56 - $100.22 an hour,California,Researcher,Senior,0.0,0.0,1.0,1.0,0.0,6
1,Slalom Consulting,Who You’ll Work With\n As a modern technology ...,80a2597defd8f97c,NaN,NaN,NaN,"Orange County, CA",Slalom Flex (Project Based) - Sr. Data Analyst,$65 - $95 an hour,California,Data analyst,Senior,0.0,1.0,0.0,0.0,0.0,5
2,Gradient,About the Company\n Gradient is on a mission t...,2d98b8b36444c36c,Full-time,NaN,NaN,"Burlingame, CA",Founding AI Strategist,"$120,000 - $200,000 a year",California,ML/AI Engineer,Not specified,0.0,0.0,0.0,1.0,0.0,4
3,Edwards Lifesciences,Imagine how your ideas and expertise can chang...,e0f49e777f057ff8,Full-time,NaN,NaN,California,"Principal, Statistical Programming Remote","$126,000 - $178,000 a year",California,Statistician,Not specified,0.0,1.0,1.0,1.0,1.0,6
4,Kittelson & Associates,Location: Oakland Orange Sacramento San Diego ...,6cc4dd9be8bd8ffa,NaN,NaN,NaN,"Oakland, CA",Senior Engineer or Planner with Focus on Trave...,"$120,000 - $150,000 a year",California,Cloud ops,Senior,0.0,1.0,1.0,1.0,0.0,0


### Analysis of job hours 

In [25]:
unique_categories_0 = final_df['jobType/0'].dropna().unique()
unique_categories_1 = final_df['jobType/1'].dropna().unique()
unique_categories_2 = final_df['jobType/2'].dropna().unique()

null_count = np.sum(final_df['jobType/0'].isnull())
print(f'{unique_categories_0}\n, the null count is {1656-null_count}')
print(f'{unique_categories_1}\n, for job1')
print(f'{unique_categories_2}\n, for job2')

type_count = np.sum([1 for i in final_df['jobType/0'] if i == 'Seasonal' or i == 'Tenured' or i=='Non-tenure' or i=='Temporary' or i =='Contract' or i=='Permanent'])
print(f'the number of jobs that might be dropped is {type_count}')
type_count_1 = np.sum([1 for i in final_df['jobType/1'] if i == 'Internship' or i == 'Freelance' or i=='Tenure track' or i=='Contract'])
print(f'the number of jobs that might be dropped is {type_count_1}')
type_count_2 = np.sum([1 for i in final_df['jobType/2'] if i == 'Internship' or i == 'Freelance' or i=='Contract' or i=='Non-tenure'])
print(f'the number of jobs that might be dropped is {type_count_2}')
#print(df['jobType/0'][4])

['Full-time' 'Contract' 'Part-time' 'Permanent' 'Temporary' 'Seasonal'
 'Tenured' 'Non-tenure']
, the null count is 1226
['Contract' 'Temporary' 'Full-time' 'Part-time' 'Internship' 'Freelance'
 'Tenure track']
, for job1
['Contract' 'Non-tenure' 'Internship' 'Full-time' 'Freelance']
, for job2
the number of jobs that might be dropped is 102
the number of jobs that might be dropped is 100
the number of jobs that might be dropped is 15


### Skills extraction

In [5]:
programming_list =  {
    "python": ["python", "Python", "PYTHON"],
    "r":["r", "R"],
    "sql": ["sql", "SQL", "Sql"],
    "julia": ["julia", "Julia", "JULIA"],
    "scala": ["scala", "Scala", "SCALA"],
    "java":["java", "Java", "JAVA"],
    "c++":["c++", "C++", "c++"],
    "matlab":["matlab", "MATLAB", "Matlab"],
    "sas":["sas", "SAS", "Sas"],
    "javascript":["javascript", "Javascript","JavaScript", "JAVASCRIPT"],
    "ruby":["ruby", "Ruby", "RUBY"],
    "php":["php", "PHP", "Php"],
    "perl":["perl", "Perl", "PERL"],
    "Swift":["Swift", "SWIFT"],
    "kotlin":["kotlin", "Kotlin", "KOTLIN"],
    "shell":["shell", "Shell", "SHELL"],
    "dart":["dart", "Dart", "DART"],
    "c":["c", "C"],
    "rust":["rust", "Rust", "RUST"]
}

In [53]:
computer_vision_software = [
    "opencv", "OpenCV", "OPENCV",
    "tensorflow", "TensorFlow", "TENSORFLOW",
    "pytorch", "PyTorch", "PYTORCH",
    "keras", "Keras", "KERAS",
    "caffe", "Caffe", "CAFFE",
    "yolo", "YOLO", "Yolo",
    "scikit-image", "Scikit-Image", "SCIKIT-IMAGE",
    "detectron2", "Detectron2", "DETECTRON2",
    "mxnet", "MXNet", "MXNET",
    "dlib", "Dlib", "DLIB",
    "deepstream", "DeepStream", "DEEPSTREAM",
    "aws rekognition", "AWS Rekognition", "AWS REKOGNITION",
    "azure", "Azure", "AZURE",
    "hugging face", "Hugging Face", "HUGGING FACE",
    "imageai", "ImageAI", "IMAGEAI",
    "torchvision", "TorchVision", "TORCHVISION",
    "fastai", "Fastai", "FASTAI"]
NLP_software = [
    "spacy", "spaCy", "SPACY",
    "nltk", "NLTK", "Nltk",
    "gensim", "Gensim", "GENSIM",
    "transformers", "Transformers", "TRANSFORMERS",
    "openai gpt", "OpenAI GPT", "OPENAI GPT",
    "bert", "BERT", "Bert",
    "fasttext", "FastText", "FASTTEXT",
    "flair", "Flair", "FLAIR",
    "corenlp", "CoreNLP", "CORENLP",
    "allennlp", "AllenNLP", "ALLENNLP",
    "fairseq", "Fairseq", "FAIRSEQ",
    "textblob", "TextBlob", "TEXTBLOB",
    "polyglot", "Polyglot", "POLYGLOT",
    "tesseract", "Tesseract", "TESSERACT",
    "opennlp", "OpenNLP", "OPENNLP",
    "deeppavlov", "DeepPavlov", "DEEPPAVLOV",
    "word2vec", "Word2Vec", "WORD2VEC"]
deep_learning_software = [
    "cntk", "CNTK", "Cntk",
    "chainer", "Chainer", "CHAINER",
    "deeplearning4j", "DeepLearning4j", "DEEPLEARNING4J",
    "onnx", "ONNX", "Onnx",
    "jax", "JAX", "Jax",
    "openai gym", "OpenAI Gym", "OPENAI GYM",
    "deepmind lab", "DeepMind Lab", "DEEPMIND LAB",
    "paddlepaddle", "PaddlePaddle", "PADDLEPADDLE",
    "neon", "Neon", "NEON",
    
]

In [54]:
database_software_list = [
    "sql", "SQL", "Sql",
    "mysql", "MySQL", "MYSQL",
    "postgresql", "PostgreSQL", "POSTGRESQL",
    "sqlite", "SQLite", "SQLITE",
    "mongodb", "MongoDB", "MONGODB",
    "oracle database", "Oracle Database", "ORACLE DATABASE",
    "microsoft sql server", "Microsoft SQL Server", "MICROSOFT SQL SERVER",
    "mariadb", "MariaDB", "MARIADB",
    "redis", "Redis", "REDIS",
    "cassandra", "Cassandra", "CASSANDRA",
    "elasticsearch", "Elasticsearch", "ELASTICSEARCH",
    "amazon dynamodb", "Amazon DynamoDB", "AMAZON DYNAMODB",
    "google bigquery", "Google BigQuery", "GOOGLE BIGQUERY",
    "snowflake", "Snowflake", "SNOWFLAKE",
    "apache hive", "Apache Hive", "APACHE HIVE",
    "couchbase", "Couchbase", "COUCHBASE",
    "neo4j", "Neo4j", "NEO4J",
]

In [55]:
data_visualisation_tools = [
    "Matplotlib",
    "matplotlib",
    "Seaborn",
    "seaborn",
    "Plotly",
    "plotly",
    "ggplot2",
    "ggplot2",
    "D3.js",
    "d3.js",
    "Tableau",
    "tableau",
    "Altair",
    "altair",
    "Bokeh",
    "bokeh",
    "Pandas (Visualization)",
    "pandas (visualization)",
    "Vega-Lite",
    "vega-lite",
    "Google Data Studio",
    "google data studio",
    "QlikView",
    "qlikview",
    "FusionCharts",
    "fusioncharts",
    "Highcharts",
    "highcharts"
]

In [56]:
General_basic_data_science_software = [
    "Orange",
    "orange",
    "KNIME",
    "knime",
    "RapidMiner",
    "rapidminer",
    "Weka",
    "weka",
    "Tableau",
    "tableau",
    "Power BI",
    "power bi",
    "Google Data Studio",
    "google data studio",
    "QlikView",
    "qlikview",
    "SAS",
    "sas",
    "Excel",
    "excel",
    "MATLAB",
    "matlab",
    "Alteryx",
    "alteryx",
    "TIBCO Spotfire",
    "tibco spotfire",
    "Zoho Analytics",
    "zoho analytics",
    "Domo",
    "domo",
    "IBM SPSS",
    "ibm spss",
    "Minitab",
    "minitab",
    "AutoML",
    "automl"
]

In [57]:
ML_areas = [
    "Supervised Learning",
    "Unsupervised Learning",
    "Reinforcement Learning",
    "Deep Learning",
    "Natural Language Processing",
    "Computer Vision",
    "Time Series Analysis",
    "Anomaly Detection",
    "Generative Models",
    "Big Data Analytics",
] 

In [58]:
Academic_background = [
    "Computer Science",
    "Information technology"
    "Mathematics",
    "Statistics",
    "Electrical Engineering",
    "Physics",
    "Medicine",
    "Econometrics",
    "Bioinformatics",
    "Neuroscience",
    "Robotics",
    "Linguistics",
    "Software Engineering",
    "Cybersecurity",
    "Economics",
    "Business",
]

In [59]:
soft_skills_ER = [
    "communication", "Communication", "COMMUNICATION",
    "teamwork", "Teamwork", "TEAMWORK",
    "problem solving", "Problem Solving", "PROBLEM SOLVING",
    "creativity", "Creativity", "CREATIVITY",
    "adaptability", "Adaptability", "ADAPTABILITY",
    "time management", "Time Management", "TIME MANAGEMENT",
    "critical thinking", "Critical Thinking", "CRITICAL THINKING",
    "leadership", "Leadership", "LEADERSHIP",
    "collaboration", "Collaboration", "COLLABORATION",
    "interpersonal skills", "Interpersonal Skills", "INTERPERSONAL SKILLS",
    "conflict resolution", "Conflict Resolution", "CONFLICT RESOLUTION",
    "emotional intelligence", "Emotional Intelligence", "EMOTIONAL INTELLIGENCE",
    "organization", "Organization", "ORGANIZATION",
    "negotiation", "Negotiation", "NEGOTIATION",
    "decision making", "Decision Making", "DECISION MAKING",
    "active listening", "Active Listening", "ACTIVE LISTENING",
    "flexibility", "Flexibility", "FLEXIBILITY",
    "attention to detail", "Attention to Detail", "ATTENTION TO DETAIL",
    "self-motivation", "Self-Motivation", "SELF-MOTIVATION",
    "motivation", "Motivation",
    "stress management", "Stress Management", "STRESS MANAGEMENT"
]
soft_skills_NER = [
    "communication", 
    "teamwork", 
    "problem solving", 
    "creativity",
    "adaptability", 
    "time management", 
    "critical thinking", 
    "leadership", 
    "collaboration", 
    "interpersonal skills", 
    "conflict resolution", 
    "emotional intelligence", 
    "organization", 
    "negotiation", 
    "decision making", 
    "active listening",
    "flexibility", 
    "attention to detail", 
    "self-motivation",
    "motivation", 
    "stress management"
    ] 

In [60]:
trained_nlp_2 = spacy.load('skills-detection-main\\trained_ner_model_2')

c:\Users\flamo\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [61]:
#HARD SKILLS patterns
#trained_nlp_2.remove_pipe("entity_ruler")
ruler3 = trained_nlp_2.add_pipe('entity_ruler', before="ner")
patterns4 = []
for key in programming_list:
    for term in programming_list[key]:
        patterns4.append({'label': key, 'pattern': term})

for term in NLP_software:
    patterns4.append({'label': "NLP_software", 'pattern': term})

for term in deep_learning_software:
    patterns4.append({'label': "Deep_learning", 'pattern': term})

for term in computer_vision_software:
    patterns4.append({'label': "Compuer_vision", 'pattern': term})

for term in database_software_list:
    patterns4.append({'label': "database_software", 'pattern': term})

for term in data_visualisation_tools:
    patterns4.append({'label': "visualisation_software", 'pattern': term})

for term in General_basic_data_science_software:
    patterns4.append({'label': "general_DS_software", 'pattern': term})

ruler3.add_patterns(patterns4)
trained_nlp_2.to_disk('trained_nlp_hard')

In [62]:
#soft skills patterns 
# trained_nlp_2.remove_pipe("entity_ruler")
trained_nlp_2.remove_pipe("entity_ruler")
ruler5 = trained_nlp_2.add_pipe('entity_ruler', before="ner")
patterns5 = []

for term in soft_skills_ER:
    patterns5.append({'label': term.lower(), 'pattern': term})

ruler5.add_patterns(patterns5)
trained_nlp_2.to_disk('trained_nlp_soft')

In [63]:
nlp_soft = spacy.load('trained_nlp_soft') # check that the patterns have been correctly added to the ER
entity_ruler = None
for name, component in nlp_soft.pipeline:
    if name == "entity_ruler":  
        entity_ruler = component
        break
patterns = entity_ruler.patterns
print("Patterns in the EntityRuler:")
for pattern in patterns:
    print(pattern)

Patterns in the EntityRuler:
{'label': 'communication', 'pattern': 'communication'}
{'label': 'communication', 'pattern': 'Communication'}
{'label': 'communication', 'pattern': 'COMMUNICATION'}
{'label': 'teamwork', 'pattern': 'teamwork'}
{'label': 'teamwork', 'pattern': 'Teamwork'}
{'label': 'teamwork', 'pattern': 'TEAMWORK'}
{'label': 'problem solving', 'pattern': 'problem solving'}
{'label': 'problem solving', 'pattern': 'Problem Solving'}
{'label': 'problem solving', 'pattern': 'PROBLEM SOLVING'}
{'label': 'creativity', 'pattern': 'creativity'}
{'label': 'creativity', 'pattern': 'Creativity'}
{'label': 'creativity', 'pattern': 'CREATIVITY'}
{'label': 'adaptability', 'pattern': 'adaptability'}
{'label': 'adaptability', 'pattern': 'Adaptability'}
{'label': 'adaptability', 'pattern': 'ADAPTABILITY'}
{'label': 'time management', 'pattern': 'time management'}
{'label': 'time management', 'pattern': 'Time Management'}
{'label': 'time management', 'pattern': 'TIME MANAGEMENT'}
{'label': '

In [64]:
nlp_hard = spacy.load('trained_nlp_hard') # check that the patterns have been correctly added to the ER
entity_ruler = None
for name, component in nlp_hard.pipeline:
    if name == "entity_ruler": 
        entity_ruler = component
        break
patterns = entity_ruler.patterns
print("Patterns in the EntityRuler:")
for pattern in patterns:
    print(pattern)

Patterns in the EntityRuler:
{'label': 'python', 'pattern': 'python'}
{'label': 'python', 'pattern': 'Python'}
{'label': 'python', 'pattern': 'PYTHON'}
{'label': 'r', 'pattern': 'r'}
{'label': 'r', 'pattern': 'R'}
{'label': 'sql', 'pattern': 'sql'}
{'label': 'sql', 'pattern': 'SQL'}
{'label': 'sql', 'pattern': 'Sql'}
{'label': 'julia', 'pattern': 'julia'}
{'label': 'julia', 'pattern': 'Julia'}
{'label': 'julia', 'pattern': 'JULIA'}
{'label': 'scala', 'pattern': 'scala'}
{'label': 'scala', 'pattern': 'Scala'}
{'label': 'scala', 'pattern': 'SCALA'}
{'label': 'java', 'pattern': 'java'}
{'label': 'java', 'pattern': 'Java'}
{'label': 'java', 'pattern': 'JAVA'}
{'label': 'c++', 'pattern': 'c++'}
{'label': 'c++', 'pattern': 'C++'}
{'label': 'c++', 'pattern': 'c++'}
{'label': 'matlab', 'pattern': 'matlab'}
{'label': 'matlab', 'pattern': 'MATLAB'}
{'label': 'matlab', 'pattern': 'Matlab'}
{'label': 'sas', 'pattern': 'sas'}
{'label': 'sas', 'pattern': 'SAS'}
{'label': 'sas', 'pattern': 'Sas'}
{'l

In [65]:
#HARD SKILLS EXTRACTION
def extract_hard_skills(text):
    text = text.replace('\n', '')
    text = text.replace('\x0c', '')
    text = re.sub(r"\d+", " ", text) 
         
    doc = nlp_hard(text)
    potential_skills = [ent.label_ for ent in doc.ents if ent.label_ not in ["EDUCATION", "SKILLS", "EXPERIENCE", "DOMAIN", "SOFT-SKILLS", "LANGUAGE", "RESPONSIBILITY"]]
    potential_skills = set(potential_skills)
    #print(potential_skills)
    final_potential_skills = list(potential_skills)
    #use similarity only for general data science terms and soft skills
    return final_potential_skills

extract_hard_skills(final_df['description'][0])


['database_software',
 'general_DS_software',
 'r',
 'scala',
 'Compuer_vision',
 'python']

In [66]:
extracted_hard_skills = []
for i in range(len(final_df['description'])):
    text = final_df['description'].iloc[i]
    hard_inter = extract_hard_skills(str(text))
    extracted_hard_skills.append(hard_inter)

In [67]:
#analyse hard skills list 
missing_inds = []
for i,k in enumerate(extracted_hard_skills):
    if k == []:
        missing_inds.append(i)
print(f'count of missng hard skills: {len(missing_inds)}, \n {missing_inds}')
print(extracted_hard_skills)

mcount of missng hard skills: 348, 
 [2, 10, 15, 16, 17, 27, 34, 35, 46, 47, 48, 49, 54, 62, 64, 66, 73, 74, 75, 77, 85, 89, 92, 95, 99, 102, 104, 112, 118, 119, 129, 132, 140, 142, 156, 158, 165, 170, 173, 177, 182, 189, 196, 200, 206, 208, 214, 225, 233, 237, 246, 250, 258, 268, 270, 272, 292, 296, 298, 299, 300, 309, 310, 314, 322, 324, 326, 328, 335, 340, 341, 342, 346, 348, 355, 360, 362, 365, 366, 375, 383, 389, 391, 392, 413, 415, 424, 428, 431, 432, 470, 471, 473, 475, 480, 484, 489, 490, 498, 509, 523, 524, 526, 527, 534, 537, 539, 542, 546, 548, 551, 553, 564, 566, 572, 576, 587, 597, 603, 605, 607, 608, 609, 612, 620, 627, 631, 632, 634, 635, 636, 638, 640, 648, 649, 650, 655, 659, 669, 684, 686, 691, 693, 699, 702, 703, 706, 709, 720, 727, 744, 745, 749, 750, 751, 758, 763, 774, 777, 782, 789, 812, 821, 823, 833, 837, 850, 853, 856, 857, 858, 861, 863, 865, 869, 872, 873, 874, 876, 879, 880, 881, 883, 886, 891, 899, 906, 912, 915, 916, 918, 924, 932, 936, 941, 944, 947, 948

In [68]:
#save the extracted list of skills to binary variables to indicate the presence or absence of a specific skill for a given job description
MILD_SKILLS = ["Compuer_vision", "NLP_software", "Deep_learning", "database_software", "visualisation_software", "general_DS_software"]
HARD_SKILLS = MILD_SKILLS + [i for i in programming_list]
print(HARD_SKILLS)
hard_skills_final = np.zeros((len(final_df['id']), len(HARD_SKILLS)))
for i in range(hard_skills_final.shape[0]):
    for j in range(hard_skills_final.shape[1]):
        #print(i,j)
        if HARD_SKILLS[j] in extracted_hard_skills[i]:
            hard_skills_final[i][j] = 1
#education_final = np.vstack([education_list, education_final])
hard_skills_df = pd.DataFrame(hard_skills_final, columns=HARD_SKILLS)
hard_skills_df.head() 

['Compuer_vision', 'NLP_software', 'Deep_learning', 'database_software', 'visualisation_software', 'general_DS_software', 'python', 'r', 'sql', 'julia', 'scala', 'java', 'c++', 'matlab', 'sas', 'javascript', 'ruby', 'php', 'perl', 'Swift', 'kotlin', 'shell', 'dart', 'c', 'rust']


,Compuer_vision,NLP_software,Deep_learning,database_software,visualisation_software,general_DS_software,python,r,sql,julia,...,javascript,ruby,php,perl,Swift,kotlin,shell,dart,c,rust
0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
final_df = pd.concat([final_df, hard_skills_df], axis=1)
final_df.head()

,company,description,id,jobType/0,jobType/1,jobType/2,location,positionName,salary,searchInput/location,...,javascript,ruby,php,perl,Swift,kotlin,shell,dart,c,rust
0,Providence,Description\n Providence is calling a Senior D...,e14d7653c80fe244,Full-time,NaN,NaN,California,Senior Data Scientist,$45.56 - $100.22 an hour,California,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Slalom Consulting,Who You’ll Work With\n As a modern technology ...,80a2597defd8f97c,NaN,NaN,NaN,"Orange County, CA",Slalom Flex (Project Based) - Sr. Data Analyst,$65 - $95 an hour,California,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Gradient,About the Company\n Gradient is on a mission t...,2d98b8b36444c36c,Full-time,NaN,NaN,"Burlingame, CA",Founding AI Strategist,"$120,000 - $200,000 a year",California,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Edwards Lifesciences,Imagine how your ideas and expertise can chang...,e0f49e777f057ff8,Full-time,NaN,NaN,California,"Principal, Statistical Programming Remote","$126,000 - $178,000 a year",California,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Kittelson & Associates,Location: Oakland Orange Sacramento San Diego ...,6cc4dd9be8bd8ffa,NaN,NaN,NaN,"Oakland, CA",Senior Engineer or Planner with Focus on Trave...,"$120,000 - $150,000 a year",California,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1656 entries, 0 to 1655
Data columns (total 43 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company                 1656 non-null   object 
 1   description             1656 non-null   object 
 2   id                      1656 non-null   object 
 3   jobType/0               1226 non-null   object 
 4   jobType/1               118 non-null    object 
 5   jobType/2               16 non-null     object 
 6   location                1656 non-null   object 
 7   positionName            1656 non-null   object 
 8   salary                  1656 non-null   object 
 9   searchInput/location    1656 non-null   object 
 10  Jobtype                 1656 non-null   object 
 11  Seniority               1656 non-null   object 
 12  Associate's Degree      1656 non-null   float64
 13  Bachelor's Degree       1656 non-null   float64
 14  Master's Degree         1656 non-null   

In [71]:
#extract soft skills 
def naive_soft_skills(text):
    text = text.replace('\n', '')
    text = text.replace('\x0c', '')
    text = re.sub(r"\d+", " ", text) 
         
    doc = nlp_soft(text)
    potential_skills = [ent.label_ for ent in doc.ents if ent.label_ not in ["EDUCATION", "SKILLS", "EXPERIENCE", "DOMAIN", "SOFT-SKILLS", "LANGUAGE", "RESPONSIBILITY"]]
    potential_skills = set(potential_skills)
    #print(potential_skills)
    final_potential_skills = list(potential_skills)
    return final_potential_skills

naive_soft_skills(final_df['description'][0])

['organization']

In [72]:
soft_skills1 = []
for i in range(len(final_df['description'])):
    text = final_df['description'].iloc[i]
    soft_inter = naive_soft_skills(str(text))
    soft_skills1.append(soft_inter)
print(soft_skills1)

[['organization'], [], [], ['collaboration', 'communication', 'attention to detail', 'leadership', 'critical thinking'], [], [], ['leadership', 'flexibility', 'organization'], ['interpersonal skills', 'communication', 'attention to detail', 'decision making'], [], ['interpersonal skills', 'communication', 'creativity'], ['collaboration', 'leadership', 'flexibility'], [], ['collaboration', 'leadership', 'organization'], [], ['communication'], ['collaboration', 'communication', 'attention to detail', 'active listening', 'organization'], ['collaboration', 'communication', 'interpersonal skills'], ['creativity', 'leadership', 'communication'], ['collaboration', 'communication'], ['collaboration', 'communication', 'organization'], [], ['creativity', 'leadership'], ['communication'], ['communication'], ['problem solving'], ['collaboration', 'communication'], [], ['leadership', 'organization'], ['creativity', 'leadership'], ['communication', 'leadership', 'organization', 'decision making'], [

In [74]:
missing_inds = []
for i,k in enumerate(soft_skills1):
    if k == []:
        missing_inds.append(i)
print(f'count of missing soft skills: {len(missing_inds)}, \n {missing_inds}')

count of missing soft skills: 223, 
 [1, 2, 4, 5, 8, 11, 13, 20, 26, 32, 63, 64, 67, 71, 74, 82, 83, 84, 90, 102, 122, 160, 169, 199, 200, 205, 211, 215, 216, 225, 229, 233, 237, 247, 258, 270, 279, 281, 291, 299, 300, 304, 309, 319, 334, 339, 346, 387, 393, 426, 428, 429, 433, 448, 452, 454, 479, 492, 505, 506, 514, 518, 570, 571, 578, 582, 586, 587, 593, 594, 596, 597, 599, 600, 603, 609, 616, 625, 629, 637, 640, 649, 658, 663, 665, 669, 673, 676, 677, 692, 696, 699, 701, 727, 728, 739, 745, 759, 771, 779, 791, 801, 811, 815, 819, 839, 861, 868, 870, 918, 928, 943, 944, 948, 957, 965, 973, 984, 991, 994, 998, 1018, 1052, 1056, 1059, 1060, 1072, 1078, 1082, 1085, 1113, 1117, 1142, 1143, 1145, 1146, 1148, 1159, 1160, 1161, 1166, 1170, 1189, 1195, 1196, 1206, 1231, 1232, 1233, 1235, 1259, 1260, 1275, 1278, 1280, 1282, 1285, 1289, 1292, 1294, 1298, 1300, 1302, 1327, 1350, 1351, 1352, 1353, 1361, 1364, 1371, 1374, 1389, 1404, 1406, 1409, 1410, 1423, 1427, 1428, 1438, 1440, 1442, 1457, 145

In [75]:
from spacy import displacy 

#1122 bachelor degree present in text and degree in entities but not bachelor nor degree extracted 

visual = final_df['description'][3]
#print(visual)
doc = nlp_soft(visual)
print(soft_skills1[3])
#print(doc.ents)
print([ent.text.lower() for ent in doc.ents])
displacy.render(doc, style = 'ent')

['collaboration', 'communication', 'attention to detail', 'leadership', 'critical thinking']
['r', 'collaboration', 'collaboration', 'leadership', 'statistics', 'mathematics', 'computer science', '6 years', 'master', 'statistics mathematics', 'computer science', '5 years', 'statistics mathematics', 'computer science', '2 years', 'powerpoint', 'communication', 'r', 'gcp', 'critical thinking', 'leadership', 'collaboration', 'attention to detail']


In [80]:
SOFT_SKILLS = [i for i in soft_skills_NER]
print(SOFT_SKILLS)
soft_skills_final = np.zeros((len(final_df['id']), len(SOFT_SKILLS)))
for i in range(soft_skills_final.shape[0]):
    for j in range(soft_skills_final.shape[1]):
        #print(i,j)
        if SOFT_SKILLS[j] in soft_skills1[i]:
            soft_skills_final[i][j] = 1
#education_final = np.vstack([education_list, education_final])
soft_skills_df = pd.DataFrame(soft_skills_final, columns=SOFT_SKILLS)
soft_skills_df.head() 

['communication', 'teamwork', 'problem solving', 'creativity', 'adaptability', 'time management', 'critical thinking', 'leadership', 'collaboration', 'interpersonal skills', 'conflict resolution', 'emotional intelligence', 'organization', 'negotiation', 'decision making', 'active listening', 'flexibility', 'attention to detail', 'self-motivation', 'motivation', 'stress management']


,communication,teamwork,problem solving,creativity,adaptability,time management,critical thinking,leadership,collaboration,interpersonal skills,...,emotional intelligence,organization,negotiation,decision making,active listening,flexibility,attention to detail,self-motivation,motivation,stress management
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
final_df = pd.concat([final_df, soft_skills_df], axis=1)
final_df.head()

,company,description,id,jobType/0,jobType/1,jobType/2,location,positionName,salary,searchInput/location,...,emotional intelligence,organization,negotiation,decision making,active listening,flexibility,attention to detail,self-motivation,motivation,stress management
0,Providence,Description\n Providence is calling a Senior D...,e14d7653c80fe244,Full-time,NaN,NaN,California,Senior Data Scientist,$45.56 - $100.22 an hour,California,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Slalom Consulting,Who You’ll Work With\n As a modern technology ...,80a2597defd8f97c,NaN,NaN,NaN,"Orange County, CA",Slalom Flex (Project Based) - Sr. Data Analyst,$65 - $95 an hour,California,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Gradient,About the Company\n Gradient is on a mission t...,2d98b8b36444c36c,Full-time,NaN,NaN,"Burlingame, CA",Founding AI Strategist,"$120,000 - $200,000 a year",California,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Edwards Lifesciences,Imagine how your ideas and expertise can chang...,e0f49e777f057ff8,Full-time,NaN,NaN,California,"Principal, Statistical Programming Remote","$126,000 - $178,000 a year",California,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,Kittelson & Associates,Location: Oakland Orange Sacramento San Diego ...,6cc4dd9be8bd8ffa,NaN,NaN,NaN,"Oakland, CA",Senior Engineer or Planner with Focus on Trave...,"$120,000 - $150,000 a year",California,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1656 entries, 0 to 1655
Data columns (total 64 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company                 1656 non-null   object 
 1   description             1656 non-null   object 
 2   id                      1656 non-null   object 
 3   jobType/0               1226 non-null   object 
 4   jobType/1               118 non-null    object 
 5   jobType/2               16 non-null     object 
 6   location                1656 non-null   object 
 7   positionName            1656 non-null   object 
 8   salary                  1656 non-null   object 
 9   searchInput/location    1656 non-null   object 
 10  Jobtype                 1656 non-null   object 
 11  Seniority               1656 non-null   object 
 12  Associate's Degree      1656 non-null   float64
 13  Bachelor's Degree       1656 non-null   float64
 14  Master's Degree         1656 non-null   

### Salary extraction

In [83]:
# extract salaries and plug them into the dataset - to be fintetuned
def extract_salary(salary_description, jobtype):
    text = re.sub('[^a-zA-Z]+', ' ', salary_description) #remove non-letters from tne strings
    numbers = re.findall(r'\$?([\d,]+\.\d+|[\d,]+)', salary_description) #look for numbers in a given format
    numbers = [float(x.replace(',', '')) for x in numbers if float(x.replace(',', '')) > 0.1] #createa a list of all the numbers present in a givne string
    #compute average yearly salaries by differentiating between types of emplyement
    if 'year' in text.split():
        if len(numbers) == 1:
            return numbers[0]
        elif len(numbers) == 2:
            average_salary1 = (float(numbers[0]) + float(numbers[1]))/2
            return average_salary1
        else:
            return None
    elif 'hour' in text.split(): 
        if jobtype == 'Part-time':
            #print(text, "hour_part_time")
            if len(numbers) == 1:
                return numbers[0]*24*52
            elif len(numbers) == 2:
                average_salary2 = (float(numbers[0]) + float(numbers[1]))*24*52/2
                return average_salary2 #run
            else:
                return None
        elif jobtype == 'Full-time':
            #print(text, "hour_full_time")
            if len(numbers) == 1:
                return numbers[0]*40*52
            elif len(numbers) == 2:
                average_salary2 = (float(numbers[0]) + float(numbers[1]))*40*52/2
                return average_salary2
            else:
                return None
        else:
            #print(jobtype)
            return None
    elif 'month' in text.split(): 
        #print(text, "month")
        if len(numbers) == 1:
            return numbers[0]*12
        elif len(numbers) == 2:
            average_salary3 = (float(numbers[0]) + float(numbers[1]))*12/2
            return average_salary3
        else:
            return None
#use the function to extract all salaries        
extracted_salary = []
nan_count = []
for i,k in enumerate(final_df['salary']):
    jobtype = final_df['jobType/0'][i]
    extracted_salary.append(extract_salary(k, jobtype))
    if (extract_salary(k, jobtype)) == "nan":
        nan_count.append(i)


final_salary = pd.DataFrame({'SALARY': extracted_salary})
final_salary.head(10)

,SALARY
0,151611.2
1,NaN
2,160000.0
3,152000.0
4,135000.0
5,250000.0
6,193750.0
7,90000.0
8,90000.0
9,86060.4


In [85]:
final_df = pd.concat([final_df, final_salary], axis=1)
final_df = final_df.drop(columns=[ "company", "location", "positionName", "description", "jobType/0","jobType/1", "jobType/2", "salary"])
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1656 entries, 0 to 1655
Data columns (total 57 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1656 non-null   object 
 1   searchInput/location    1656 non-null   object 
 2   Jobtype                 1656 non-null   object 
 3   Seniority               1656 non-null   object 
 4   Associate's Degree      1656 non-null   float64
 5   Bachelor's Degree       1656 non-null   float64
 6   Master's Degree         1656 non-null   float64
 7   Degree                  1656 non-null   float64
 8   Doctorate               1656 non-null   float64
 9   EXPERIENCE              1656 non-null   int64  
 10  Compuer_vision          1656 non-null   float64
 11  NLP_software            1656 non-null   float64
 12  Deep_learning           1656 non-null   float64
 13  database_software       1656 non-null   float64
 14  visualisation_software  1656 non-null   

In [86]:
final_df.to_excel('refined_NLP_final3.xlsx', index=False)